In [3]:
from tensorflow.keras.preprocessing.text import Tokenizer, one_hot

In [5]:
import json
with open('data/dictionary.json', 'r') as file:
    dictionary = json.load(file)

In [6]:
import pandas as pd
vocab_size=len(dictionary) 
comments = pd.read_csv('./data/df_pre.csv')['comment']
comments = comments.apply(one_hot, n=vocab_size)

In [7]:
maxlen=-1
for sentence in comments:    
    if(maxlen<len(sentence)):     
        maxlen=len(sentence)
print("The maximum number of words in any document is : ",maxlen)

The maximum number of words in any document is :  439


In [9]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
comments = pad_sequences(comments, maxlen=maxlen, padding='post', value=0.0)

In [18]:
from tensorflow.keras.layers import Input, Embedding, Flatten, Dense, Dropout
from tensorflow.keras.models import Model

w_input = Input(shape=(maxlen,), dtype='float64')

# creating the embedding
word_embedding=Embedding(input_dim=vocab_size,output_dim=20,input_length=maxlen)(w_input)

word_vec = Flatten()(word_embedding) # flatten

x = Dense(500, activation='relu')(word_vec)
x = Dropout(0.5)(x)
x = Dense(100, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(10, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(1, activation='sigmoid', name='output')(x)

embed_model = Model([w_input], output) # combining all into a Keras model

In [19]:
embed_model.summary()

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 439)]             0         
_________________________________________________________________
embedding_4 (Embedding)      (None, 439, 20)           2831620   
_________________________________________________________________
flatten_4 (Flatten)          (None, 8780)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 500)               4390500   
_________________________________________________________________
dropout_6 (Dropout)          (None, 500)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 100)               50100     
_________________________________________________________________
dropout_7 (Dropout)          (None, 100)               0   

In [20]:
from tensorflow import keras
embed_model.compile(
    loss=keras.losses.BinaryCrossentropy(from_logits=False),
    optimizer=keras.optimizers.Adam(),
    metrics=[keras.metrics.BinaryAccuracy(), keras.metrics.Precision(), keras.metrics.Recall()]
    )

In [21]:
Y = pd.read_csv('./data/df_pre.csv')['isSarcastic']
history = embed_model.fit(comments, Y, batch_size=64, epochs=5, validation_split=0.2)

Epoch 1/5
12635/12635 [==============================] - 362s 29ms/step - loss: 0.6588 - binary_accuracy: 0.5892 - precision: 0.6430 - recall: 0.3300 - val_loss: 0.5892 - val_binary_accuracy: 0.6831 - val_precision: 0.7351 - val_recall: 0.6525
Epoch 2/5
12635/12635 [==============================] - 345s 27ms/step - loss: 0.5994 - binary_accuracy: 0.6850 - precision: 0.7238 - recall: 0.5764 - val_loss: 0.5930 - val_binary_accuracy: 0.6785 - val_precision: 0.7617 - val_recall: 0.5953
Epoch 3/5
12635/12635 [==============================] - 361s 29ms/step - loss: 0.5776 - binary_accuracy: 0.7026 - precision: 0.7339 - recall: 0.6144 - val_loss: 0.5901 - val_binary_accuracy: 0.6826 - val_precision: 0.7286 - val_recall: 0.6636
Epoch 4/5
12635/12635 [==============================] - 332s 26ms/step - loss: 0.5536 - binary_accuracy: 0.7229 - precision: 0.7466 - recall: 0.6553 - val_loss: 0.6043 - val_binary_accuracy: 0.6768 - val_precision: 0.7302 - val_recall: 0.6437
Epoch 5/5
  457/12635 [>

KeyboardInterrupt: 